In [ ]:
!pip install seqeval
!pip install transformers
!pip install datasets

     |████████████████████████████████| 43 kB 1.1 MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16180 sha256=10bd5d73336fd80e939b8a389ecf38ecc845f254a7ef196f7e6b330fbd24c709
  Stored in directory: /root/.cache/pip/wheels/05/96/ee/7cac4e74f3b19e3158dce26a20a1c86b3533c43ec72a549fd7
Successfully built seqeval
     |████████████████████████████████| 3.5 MB 4.8 MB/s 
     |████████████████████████████████| 67 kB 4.3 MB/s 
     |████████████████████████████████| 596 kB 30.5 MB/s 
     |████████████████████████████████| 6.5 MB 28.0 MB/s 
     |████████████████████████████████| 895 kB 33.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 311 kB 5.2 MB/s 
     |████████████████████████████████| 212 kB 45.5 MB/s 
     |████████████████████████████████| 1.1 MB 43.4 MB/s 
     |████████████████████████████████| 134 kB 42

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token


In [ ]:
!git config --global credential.helper store

In [ ]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.3.4-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-470
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.


In [ ]:
task = "ner" # Should be one of "ner", "pos" or "chunk"
model_checkpoint ="dmis-lab/biobert-base-cased-v1.2"
batch_size = 8

In [ ]:
#We extract the words along with their named entities into an array
def split_text_label(filename):
    f = open(filename)
    split_labeled_text = []
    sentence = []
    for line in f:
        
        if len(line)==0 or line.startswith('-DOCSTART') or line[0]=="\n":
            if len(sentence) > 0:
                split_labeled_text.append(sentence)
                sentence = []
                continue
        splits = line.split('\t')
        sentence.append([splits[0],splits[-1].rstrip("\n")])
    if len(sentence) > 0:
        split_labeled_text.append(sentence)
        sentence = []
    return split_labeled_text

In [ ]:
import os
split_train = split_text_label("/content/train.tsv")
split_test = split_text_label("/content/test.tsv")
split_val = split_text_label("/content/devel.tsv")
split_gaz = split_text_label("/content/final_gazetteer.txt")

In [ ]:
split_train= split_train+split_gaz

In [ ]:
split_train

[[['Intraocular', 'O'],
  ['pressure', 'O'],
  ['in', 'O'],
  ['genetically', 'B-SO'],
  ['distinct', 'O'],
  ['mice', 'B-Taxon'],
  [':', 'O'],
  ['an', 'O'],
  ['update', 'O'],
  ['and', 'O'],
  ['strain', 'O'],
  ['survey', 'O']],
 [['Abstract', 'O']],
 [['Background', 'O']],
 [['Little', 'O'],
  ['is', 'O'],
  ['known', 'O'],
  ['about', 'O'],
  ['genetic', 'B-SO'],
  ['factors', 'O'],
  ['affecting', 'O'],
  ['intraocular', 'O'],
  ['pressure', 'O'],
  ['(', 'O'],
  ['IOP', 'O'],
  [')', 'O'],
  ['in', 'O'],
  ['mice', 'B-Taxon'],
  ['and', 'O'],
  ['other', 'O'],
  ['mammals', 'B-Taxon'],
  ['.', 'O']],
 [['The', 'O'],
  ['purpose', 'O'],
  ['of', 'O'],
  ['this', 'O'],
  ['study', 'O'],
  ['was', 'O'],
  ['to', 'O'],
  ['determine', 'O'],
  ['the', 'O'],
  ['IOPs', 'O'],
  ['of', 'O'],
  ['genetically', 'B-SO'],
  ['distinct', 'O'],
  ['mouse', 'B-Taxon'],
  ['strains', 'O'],
  [',', 'O'],
  ['assess', 'O'],
  ['the', 'O'],
  ['effects', 'O'],
  ['of', 'O'],
  ['factors', 'O'],


In [ ]:
len(split_train)

23521

In [ ]:
dict_new= {#GENE
           'B-Gene_or_gene_product': 'B-GENE','I-Gene_or_gene_product':'I-GENE',
           'E-Gene_or_gene_product':'E-GENE','S-Gene_or_gene_product':'S-GENE',
           'B-GO':'B-GENE','I-GO':'I-GENE','E-GO' :'E-GENE' ,'S-GO':'S-GENE',
            "B-GENE":"B-GENE", "I-GENE":"I-GENE", "E-GENE":"E-GENE","S-GENE":"S-GENE",
           #Chemical
            "B-CHEBI":"B-Chemical", "I-CHEBI":"I-Chemical","E-CHEBI":"E-Chemical","S-CHEBI":"S-Chemical",
            "B-Simple_chemical":"B-Chemical", "I-Simple_chemical":"I-Chemical","E-Simple_chemical":"E-Chemical",
            "S-Simple_chemical":"S-Chemical", "B-Chemical":"B-Chemical", "I-Chemical":"I-Chemical","E-Chemical":"E-Chemical",
            "S-Chemical":"S-Chemical",
           #organism
            "B-Organism_subdivision": "B-Organism","I-Organism_subdivision":"I-Organism",
            "E-Organism_subdivision":"E-Organism","S-Organism_subdivision" :"S-Organism",
            "B-Organism":"B-Organism","I-Organism":"I-Organism","E-Organism":"E-Organism","S-Organism":"S-Organism",
            #cell
            "B-Cellular_component": "B-Cell" ,"E-Cellular_component":"E-Cell","I-Cellular_component":"I-Cell",
            "S-Cellular_component":"S-Cell","B-CL": "B-Cell" ,"E-CL":"E-Cell",
            "I-CL":"I-Cell", "S-CL":"S-Cell", "B-Cell":"B-Cell", "I-Cell":"I-Cell", "E-Cell":"E-Cell", "S-Cell":"S-Cell",
            #tissue
            "B-Multi-tissue_structure": "B-Tissue","I-Multi-tissue_structure": "I-Tissue",
            "E-Multi-tissue_structure": "E-Tissue","S-Multi-tissue_structure": "S-Tissue",
            "B-Tissue":"B-Tissue", "I-Tissue":"I-Tissue","E-Tissue":"E-Tissue","S-Tissue":"S-Tissue",
            #pathology
            "B-Pathological_formation":"B-Pathology","I-Pathological_formation":"I-Pathology",
            "E-Pathological_formation":"E-Pathology","S-Pathological_formation":"S-Pathology",
            "B-Pathology":"B-Pathology","I-Pathology":"I-Pathology","E-Pathology":"E-Pathology",
            "S-Pathology":"S-Pathology",
            #Anatomy
            "B-Anatomical_system":"B-Anatomy", "I-Anatomical_system":"I-Anatomy",
            "E-Anatomical_system":"E-Anatomy","S-Anatomical_system":"S-Anatomy",
            "B-Developing_anatomical_structure" :"B-Anatomy", "I-Developing_anatomical_structure" :"I-Anatomy",
            "E-Developing_anatomical_structure" :"E-Anatomy", "S-Developing_anatomical_structure" :"S-Anatomy",
            "B-Immaterial_anatomical_entity" :"B-Anatomy","I-Immaterial_anatomical_entity" :"I-Anatomy",
            "E-Immaterial_anatomical_entity" :"E-Anatomy","S-Immaterial_anatomical_entity" :"S-Anatomy",
            "B-Anatomy":"B-Anatomy","I-Anatomy":"I-Anatomy","E-Anatomy":"E-Anatomy","S-Anatomy":"S-Anatomy",
            
            'B-Disease': 'B-Disease','I-Disease': 'I-Disease','E-Disease': 'E-Disease','S-Disease': 'S-Disease',
            'B-Protein':'B-Protein', 'I-Protein':'I-Protein','E-Protein':'E-Protein','S-Protein':'S-Protein',
            "B-Cancer":"B-Cancer","I-Cancer":"I-Cancer","E-Cancer":"E-Cancer","S-Cancer":"S-Cancer",
            'B-Organ':'B-Organ', 'I-Organ':'I-Organ','E-Organ':'E-Organ','S-Organ':'S-Organ',
            'B-Complex':'B-Complex', 'I-Complex':'I-Complex', 'E-Complex':'E-Complex','S-Complex':'S-Complex',
            'B-Taxon':'B-Taxon', 'I-Taxon':'I-Taxon', 'E-Taxon':'E-Taxon','S-Taxon':'S-Taxon',
            #protein
            'B-GGP': 'B-Protein', 'I-GGP':'I-Protein', 'E-GGP':'E-Protein', 'S-GGP':'S-Protein',
            #Sequence
            "B-SO":"B-Sequence","I-SO":"I-Sequence","E-SO":"E-Sequence","S-SO":"S-Sequence",
            #to 0
            'B-Amino_acid':"O",'I-Amino_acid':"O",'E-Amino_acid':"O",'S-Amino_acid':"O",
            'B-Regulon-operon':"O", 'I-Regulon-operon':"O",'E-Regulon-operon':"O",'S-Regulon-operon':"O",
            "B-Organism_substance":"O","I-Organism_substance":"O","E-Organism_substance":"O","S-Organism_substance":"O"
            
          }

In [ ]:
'''
It replaces the tags to match and normalize only one standard way.
It also creates the list of lists with format [w,t]. Modify accordingly. remove the dict_new object

'''
#replace all not "O" with the value in the above dictionary
#Optional for the biomedical


'\nIt replaces the tags to match and normalize only one standard way.\nIt also creates the list of lists with format [w,t]. Modify accordingly. remove the dict_new object\n\n'

In [ ]:
lista_split_train=[]
lista_split_test=[]
lista_split_val=[]
def replace_group(group, lista):

  for split in group:
      temp_list=[]
      #print(split)
      for word, tag in split:
              
          if tag != "O":
              for k,v in dict_new.items():
                  if tag == k:
                      temp_list.append([word, tag.replace(tag, v)])
                  #else:
                  #    temp_list.append([word, tag.replace(tag, "O")])
          elif tag == "O":
              temp_list.append([word,tag])
          
      lista.append(temp_list)
      temp_list =[]
  return lista

In [ ]:
lista_train=replace_group(split_train, lista_split_train)
lista_test=replace_group(split_test, lista_split_test)
lista_val=replace_group(split_val, lista_split_val)

In [ ]:
lista_train

[[['Intraocular', 'O'],
  ['pressure', 'O'],
  ['in', 'O'],
  ['genetically', 'B-Sequence'],
  ['distinct', 'O'],
  ['mice', 'B-Taxon'],
  [':', 'O'],
  ['an', 'O'],
  ['update', 'O'],
  ['and', 'O'],
  ['strain', 'O'],
  ['survey', 'O']],
 [['Abstract', 'O']],
 [['Background', 'O']],
 [['Little', 'O'],
  ['is', 'O'],
  ['known', 'O'],
  ['about', 'O'],
  ['genetic', 'B-Sequence'],
  ['factors', 'O'],
  ['affecting', 'O'],
  ['intraocular', 'O'],
  ['pressure', 'O'],
  ['(', 'O'],
  ['IOP', 'O'],
  [')', 'O'],
  ['in', 'O'],
  ['mice', 'B-Taxon'],
  ['and', 'O'],
  ['other', 'O'],
  ['mammals', 'B-Taxon'],
  ['.', 'O']],
 [['The', 'O'],
  ['purpose', 'O'],
  ['of', 'O'],
  ['this', 'O'],
  ['study', 'O'],
  ['was', 'O'],
  ['to', 'O'],
  ['determine', 'O'],
  ['the', 'O'],
  ['IOPs', 'O'],
  ['of', 'O'],
  ['genetically', 'B-Sequence'],
  ['distinct', 'O'],
  ['mouse', 'B-Taxon'],
  ['strains', 'O'],
  [',', 'O'],
  ['assess', 'O'],
  ['the', 'O'],
  ['effects', 'O'],
  ['of', 'O'],
  

In [ ]:
'''We build the vocabulary for all unique words and unique labels (nes are the labels)
'''

'We build the vocabulary for all unique words and unique labels (nes are the labels)\n'

In [ ]:
words = set()
tags = set()
# words and labels
for data in [lista_train, lista_test, lista_val]:
    for labeled_text in data:
        for word, label in labeled_text:
            tags.add(label)
            words.add(word.lower())
            
tags_vals= list(tags)

In [ ]:
print(tags_vals)
print(len(tags_vals))
label_list=sorted(tags_vals)

['I-Protein', 'I-Cell', 'I-Chemical', 'B-GENE', 'B-Cell', 'B-Chemical', 'I-Sequence', 'I-Taxon', 'O', 'B-Taxon', 'B-Protein', 'B-Sequence', 'I-GENE']
13


In [ ]:
#map words to index and tags to index
#words2index = {w:i for i,w in enumerate(words)}
tags2index = {t:i for i,t in enumerate(tags_vals)}
#print(words2index['london'])
print(tags2index)

{'I-Protein': 0, 'I-Cell': 1, 'I-Chemical': 2, 'B-GENE': 3, 'B-Cell': 4, 'B-Chemical': 5, 'I-Sequence': 6, 'I-Taxon': 7, 'O': 8, 'B-Taxon': 9, 'B-Protein': 10, 'B-Sequence': 11, 'I-GENE': 12}


In [ ]:
tag2name={tags2index[key] : key for key in tags2index.keys()}
print(tag2name)

{0: 'I-Protein', 1: 'I-Cell', 2: 'I-Chemical', 3: 'B-GENE', 4: 'B-Cell', 5: 'B-Chemical', 6: 'I-Sequence', 7: 'I-Taxon', 8: 'O', 9: 'B-Taxon', 10: 'B-Protein', 11: 'B-Sequence', 12: 'I-GENE'}


In [ ]:
'''
This next function replace the tag for a number, adds the id for the Hugging dataset.
'''

'\nThis next function replace the tag for a number, adds the id for the Hugging dataset.\n'

In [ ]:
lista_big_train=[]
lista_big_test=[]
lista_big_val=[]

def replace_tag(lista_rep, big_lista):

  id=""
  ner_tags=[]
  for i,lista in enumerate(lista_rep):
    temp_token=[]
    temp_tags=[]
    for word,tag in lista:
      temp_token.append(word)
      id= i
      for k,v in tags2index.items():
        if tag == k:
          temp_tags.append(v)
        
    #tokens.append(temp_token)
    #ner_tag.append(temp_tags)
    big_lista.append([id, temp_token, temp_tags])
  return big_lista

In [ ]:
lista_big_train = replace_tag(lista_train, lista_big_train)
lista_big_test = replace_tag(lista_test, lista_big_test)
lista_big_val = replace_tag(lista_val, lista_big_val)

In [ ]:
print(len(lista_big_val))
print(len(lista_val))

3730
3730


In [ ]:
import pandas as pd
df_val = pd.DataFrame(lista_big_val, columns=["id", "tokens", "ner_tags"])
df_train = pd.DataFrame(lista_big_train, columns=["id", "tokens", "ner_tags"])
df_test = pd.DataFrame(lista_big_test, columns=["id", "tokens", "ner_tags"])

In [ ]:
from datasets import Dataset

dataset_val = Dataset.from_pandas(df_val)
dataset_train=Dataset.from_pandas(df_train)
dataset_test=Dataset.from_pandas(df_test)

In [ ]:
dataset_train.features

{'id': Value(dtype='int64', id=None),
 'ner_tags': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None),
 'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)}

In [ ]:
dataset_train

Dataset({
    features: ['id', 'tokens', 'ner_tags'],
    num_rows: 23521
})

In [ ]:
import datasets
datasets= datasets.DatasetDict({"train": dataset_train, "validation": dataset_val, "test": dataset_test})

In [ ]:
datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 23521
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 3730
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 7425
    })
})

In [ ]:
datasets["validation"][0]

{'id': 0,
 'ner_tags': [8, 8, 8, 8, 8, 10, 0, 11, 8, 8, 11, 6, 6, 8, 8, 8, 9],
 'tokens': ['Cloning',
  'and',
  'characterization',
  'of',
  'the',
  'mouse',
  'Mcoln1',
  'gene',
  'reveals',
  'an',
  'alternatively',
  'spliced',
  'transcript',
  'not',
  'seen',
  'in',
  'humans']}

In [ ]:
datasets["train"].features[f"ner_tags"]

Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None)

In [ ]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(datasets["train"])

,id,tokens,ner_tags
0,18328,"[positive, regulation, of, protein, localization, to, synapse]","[4, 1, 1, 1, 1, 1, 1]"
1,10417,"[Testis, sections, from, adult, mice, stained, with, antibody, to, androgen, receptor, (, red, ), and, counterstained, with, DAPI, (, blue, ), .]","[8, 8, 8, 8, 9, 8, 8, 3, 8, 10, 0, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8]"
2,6822,"[(, A, ), A, representative, example, of, the, genotyping, of, blastocyst, stage, embryos, .]","[8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8]"
3,4870,"[In, contrast, ,, neurite, outgrowth, of, Bax, -, /, -, DRG, neurons, was, significantly, less, (, see, Figure, 4G, ), than, that, of, either, DRG, from, TauEWS, -, Pea3, /, +, Isl1Cre, /, +, embryos, cultured, in, the, absence, of, neurotrophic, support, (, see, Figure, 4D, ), or, Bax, -, /, -, DRG, neurons, cultured, in, the, presence, of, neurotrophic, support, (, see, Figure, 4H, and, 4I, ), .]","[8, 8, 8, 8, 8, 8, 10, 8, 8, 8, 8, 4, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 10, 8, 11, 8, 8, 11, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 10, 8, 8, 8, 8, 4, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8]"
4,17211,"[cellular, response, to, gold, (, 3+, )]","[3, 12, 12, 12, 12, 12, 12]"
5,5758,"[B6, mice, (, blue, line, on, left, panel, ), are, more, sensitive, to, 1, mM, and, 3, mM, QHCl, than, are, D2, mice, (, red, line, on, left, panel, ), in, brief, access, taste, tests, .]","[8, 9, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8]"
6,19874,[],[]
7,7540,"[This, selection, scheme, was, used, at, each, generation, until, a, N4, male, was, identified, as, homozygous, HG, for, all, markers, typed, outside, MMU2, .]","[8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8]"
8,1361,"[These, 14, sequences, also, revealed, one, cDNA, where, the, poly, (, A, ), tail, was, added, 27, bp, before, the, stop, codon, ,, and, another, where, an, intron, was, spliced, out, of, the, 3, ', UTR, ,, contrary, to, the, conventional, stereotype, of, olfactory, receptor, gene, structure, .]","[8, 8, 11, 8, 8, 8, 11, 8, 8, 11, 6, 6, 6, 8, 8, 8, 8, 11, 8, 8, 11, 6, 8, 8, 8, 8, 8, 11, 8, 8, 8, 8, 8, 11, 6, 6, 8, 8, 8, 8, 8, 8, 8, 8, 8, 11, 8, 8]"
9,13479,"[D-2-Aminoglutaric, acid]","[5, 2]"


In [ ]:
#preprocessing
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, model_max_length=512)
#tokenizer= RobertaTokenizerFast.from_pretrained(model_checkpoint, add_prefix_space=True)

In [ ]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

In [ ]:
tokenizer("Hello, this is one sentence!")

{'input_ids': [101, 19082, 117, 1142, 1110, 1141, 5650, 106, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
tokenizer(["Hello", ",", "this", "is", "one", "sentence", "split", "into", "words", "."], is_split_into_words=True)

{'input_ids': [101, 19082, 117, 1142, 1110, 1141, 5650, 3325, 1154, 1734, 119, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
example = datasets["train"][10]
print(example["tokens"])

['Albino', 'C57BL', '/', '6J', 'mice', 'homozygous', 'for', 'a', 'tyrosinase', 'mutation', '(', 'Tyrc', '-', '2J', ')', 'have', 'higher', 'IOPs', 'than', 'their', 'pigmented', 'counterparts', '.']


In [ ]:
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print(tokens)

['[CLS]', 'al', '##bino', 'c', '##5', '##7', '##b', '##l', '/', '6', '##j', 'mice', 'ho', '##mo', '##zy', '##go', '##us', 'for', 'a', 't', '##yr', '##os', '##inas', '##e', 'mutation', '(', 't', '##yr', '##c', '-', '2', '##j', ')', 'have', 'higher', 'i', '##ops', 'than', 'their', 'pig', '##mented', 'counterparts', '.', '[SEP]']


In [ ]:
len(example[f"{task}_tags"]), len(tokenized_input["input_ids"])

(23, 44)

In [ ]:
print(tokenized_input.word_ids())

[None, 0, 0, 1, 1, 1, 1, 1, 2, 3, 3, 4, 5, 5, 5, 5, 5, 6, 7, 8, 8, 8, 8, 8, 9, 10, 11, 11, 11, 12, 13, 13, 14, 15, 16, 17, 17, 18, 19, 20, 20, 21, 22, None]


In [ ]:
word_ids = tokenized_input.word_ids()
aligned_labels = [-100 if i is None else example[f"{task}_tags"][i] for i in word_ids]
print(len(aligned_labels), len(tokenized_input["input_ids"]))

44 44


In [ ]:
label_all_tokens = False

In [ ]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"{task}_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
tokenize_and_align_labels(datasets['train'][:1])

{'input_ids': [[101, 1107, 4487, 13335, 5552, 2997, 1107, 21989, 4966, 14105, 131, 1126, 11984, 1105, 10512, 5980, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[-100, 8, -100, -100, -100, 8, 8, 11, 8, 9, 8, 8, 8, 8, 8, 8, -100]]}

In [ ]:
tokenized_datasets = datasets.map(tokenize_and_align_labels, batched=True)

  0%|          | 0/24 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/8 [00:00<?, ?ba/s]

In [ ]:
#finetuning
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))

Some weights of the model checkpoint at dmis-lab/biobert-base-cased-v1.2 were not used when initializing BertForTokenClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initi

In [ ]:
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-finetuned-{task}-CRAFT_Gazetteer1",
    evaluation_strategy = "epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=4,
    weight_decay=0.01,
    save_steps=7000,
    save_total_limit=4,
    push_to_hub=True,
)

In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [ ]:
from datasets import load_dataset, load_metric
metric = load_metric("seqeval")

In [ ]:
labels = [label_list[i] for i in example[f"{task}_tags"]]
metric.compute(predictions=[labels], references=[labels])

{'GENE': {'f1': 1.0, 'number': 4, 'precision': 1.0, 'recall': 1.0},
 'Protein': {'f1': 1.0, 'number': 1, 'precision': 1.0, 'recall': 1.0},
 'Sequence': {'f1': 1.0, 'number': 1, 'precision': 1.0, 'recall': 1.0},
 'Taxon': {'f1': 1.0, 'number': 2, 'precision': 1.0, 'recall': 1.0},
 'overall_accuracy': 1.0,
 'overall_f1': 1.0,
 'overall_precision': 1.0,
 'overall_recall': 1.0}

In [ ]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Cloning https://huggingface.co/StivenLancheros/biobert-base-cased-v1.2-finetuned-ner-CRAFT_Gazetteer1 into local empty directory.


In [ ]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: tokens, ner_tags, id.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 23521
  Num Epochs = 4
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 11764


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.122100,0.146613,0.829529,0.795588,0.812204,0.966092
2,0.056800,0.152486,0.828910,0.830647,0.829778,0.968379
3,0.037000,0.184426,0.841067,0.822224,0.831539,0.968469
4,0.023800,0.195383,0.845077,0.828167,0.836536,0.969738


The following columns in the evaluation set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: tokens, ner_tags, id.
***** Running Evaluation *****
  Num examples = 3730
  Batch size = 8
The following columns in the evaluation set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: tokens, ner_tags, id.
***** Running Evaluation *****
  Num examples = 3730
  Batch size = 8
Saving model checkpoint to biobert-base-cased-v1.2-finetuned-ner-CRAFT_Gazetteer1/checkpoint-7000
Configuration saved in biobert-base-cased-v1.2-finetuned-ner-CRAFT_Gazetteer1/checkpoint-7000/config.json
Model weights saved in biobert-base-cased-v1.2-finetuned-ner-CRAFT_Gazetteer1/checkpoint-7000/pytorch_model.bin
tokenizer config file saved in biobert-base-cased-v1.2-finetuned-ner-CRAFT_Gazetteer1/checkpoint-7000/tokenizer_config.json
Special tokens file saved in biobert-base-cased-v1.2-finetuned-ner-CRAFT_Gazetteer1/chec

TrainOutput(global_step=11764, training_loss=0.07489939742816301, metrics={'train_runtime': 3350.95, 'train_samples_per_second': 28.077, 'train_steps_per_second': 3.511, 'total_flos': 2817339374605638.0, 'train_loss': 0.07489939742816301, 'epoch': 4.0})

In [ ]:
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: tokens, ner_tags, id.
***** Running Evaluation *****
  Num examples = 3730
  Batch size = 8


{'epoch': 4.0,
 'eval_accuracy': 0.9697375254684192,
 'eval_f1': 0.836536385229972,
 'eval_loss': 0.19538277387619019,
 'eval_precision': 0.8450765673399939,
 'eval_recall': 0.8281670870922229,
 'eval_runtime': 46.8254,
 'eval_samples_per_second': 79.658,
 'eval_steps_per_second': 9.973}

In [ ]:
predictions, labels, _ = trainer.predict(tokenized_datasets["validation"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

The following columns in the test set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: tokens, ner_tags, id.
***** Running Prediction *****
  Num examples = 3730
  Batch size = 8


{'Cell': {'f1': 0.7168508287292819,
  'number': 1504,
  'precision': 0.7456896551724138,
  'recall': 0.6901595744680851},
 'Chemical': {'f1': 0.6614173228346456,
  'number': 327,
  'precision': 0.6818181818181818,
  'recall': 0.6422018348623854},
 'GENE': {'f1': 0.8276093147930977,
  'number': 12031,
  'precision': 0.8341072182355425,
  'recall': 0.8212118693375446},
 'Protein': {'f1': 0.8962700789545333,
  'number': 1858,
  'precision': 0.9068870523415978,
  'recall': 0.8858988159311088},
 'Sequence': {'f1': 0.7988962830709301,
  'number': 3116,
  'precision': 0.8082101806239738,
  'recall': 0.7897946084724005},
 'Taxon': {'f1': 0.9118976733928769,
  'number': 4553,
  'precision': 0.9156333038086802,
  'recall': 0.9081924006149792},
 'overall_accuracy': 0.9697375254684192,
 'overall_f1': 0.836536385229972,
 'overall_precision': 0.8450765673399939,
 'overall_recall': 0.8281670870922229}

In [ ]:
trainer.push_to_hub()

Saving model checkpoint to biobert-base-cased-v1.2-finetuned-ner-CRAFT_Gazetteer1
Configuration saved in biobert-base-cased-v1.2-finetuned-ner-CRAFT_Gazetteer1/config.json
Model weights saved in biobert-base-cased-v1.2-finetuned-ner-CRAFT_Gazetteer1/pytorch_model.bin
tokenizer config file saved in biobert-base-cased-v1.2-finetuned-ner-CRAFT_Gazetteer1/tokenizer_config.json
Special tokens file saved in biobert-base-cased-v1.2-finetuned-ner-CRAFT_Gazetteer1/special_tokens_map.json
Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file pytorch_model.bin:   0%|          | 3.34k/411M [00:00<?, ?B/s]

Upload file runs/Feb26_13-25-24_5bb0d710f1de/events.out.tfevents.1645885332.5bb0d710f1de.599.2: 100%|#########…

Upload file runs/Feb26_13-25-24_5bb0d710f1de/events.out.tfevents.1645881935.5bb0d710f1de.599.0:  35%|###5     …

To https://huggingface.co/StivenLancheros/biobert-base-cased-v1.2-finetuned-ner-CRAFT_Gazetteer1
   9447c49..cbbb5fc  main -> main

Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Token Classification', 'type': 'token-classification'}, 'metrics': [{'name': 'Precision', 'type': 'precision', 'value': 0.8450765673399939}, {'name': 'Recall', 'type': 'recall', 'value': 0.8281670870922229}, {'name': 'F1', 'type': 'f1', 'value': 0.836536385229972}, {'name': 'Accuracy', 'type': 'accuracy', 'value': 0.9697375254684192}]}
To https://huggingface.co/StivenLancheros/biobert-base-cased-v1.2-finetuned-ner-CRAFT_Gazetteer1
   cbbb5fc..cfad787  main -> main



'https://huggingface.co/StivenLancheros/biobert-base-cased-v1.2-finetuned-ner-CRAFT_Gazetteer1/commit/cbbb5fc1ec8ce699a87b199bf3e9707bfc8d68ec'

In [ ]:
{
  'I-Protein': 0, 
 'B-Sequence': 1, 
 'B-Cell': 2, 
 'I-Chemical': 3, 
 'I-Cell': 4, 
 'B-GENE': 5, 
 'I-GENE': 6, 
 'I-Taxon': 7, 
 'B-Taxon': 8, 
 'O': 9, 
 'B-Protein': 10, 
 'I-Sequence': 11, 
 'B-Chemical': 12
 }

{
  0: 'I-Protein', 
 1: 'B-Sequence', 
 2: 'B-Cell', 
 3: 'I-Chemical', 
 4: 'I-Cell', 
 5: 'B-GENE', 
 6: 'I-GENE', 
 7: 'I-Taxon', 
 8: 'B-Taxon', 
 9: 'O', 
 10: 'B-Protein', 
 11: 'I-Sequence', 
 12: 'B-Chemical'
 }


{0: 'I-Protein',
 1: 'B-Sequence',
 2: 'B-Cell',
 3: 'I-Chemical',
 4: 'I-Cell',
 5: 'B-GENE',
 6: 'I-GENE',
 7: 'I-Taxon',
 8: 'B-Taxon',
 9: 'O',
 10: 'B-Protein',
 11: 'I-Sequence',
 12: 'B-Chemical'}